In [ ]:
import base64
import os
from tkinter import filedialog
from openai import OpenAI
import pyperclip
from PIL import Image

In [2]:
def encode_image_to_base64(file_path):
    with open(file_path, "rb") as image_file:
        image_data = image_file.read()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        return f"data:image/jpeg;base64,{base64_image}"

In [ ]:
MODEL="gpt-4o"
client = OpenAI(api_key="API")

In [4]:
from pathlib import Path

In [5]:
data_root = Path("")

In [ ]:
from tqdm import tqdm
import PIL
from concurrent.futures import ThreadPoolExecutor, as_completed

def resize(img_path: Path, destination_folder: Path) -> None:
    destination_path = (destination_folder / img_path.name).with_suffix(".jpg")
    if destination_path.exists():
        return

    try:
        img = Image.open(img_path)
    except PIL.UnidentifiedImageError as e:
        print(e)
        return

    imgResize = img.resize((428*2, 240*2))
    imgResize.save(
        tmp_name := (destination_folder / img_path.name).with_suffix(".tmp"), format="jpeg", quality=90)
    tmp_name.rename(destination_path)


# crop images
with ThreadPoolExecutor() as executor:
    for source_folder in (progress := tqdm(list((data_root/"videos").glob("VID*")))):
        progress.set_description(source_folder.name)
        destination_folder = data_root / "videos_cropped" / source_folder.name
        destination_folder.mkdir(parents=True, exist_ok=True)

        frames = list(source_folder.glob("*.png"))
        futures = [
            executor.submit(resize, img_path=item, destination_folder=destination_folder)
            for item in frames
        ]

        for _ in as_completed(futures):
            pass

In [ ]:
# Select images
destination_folder = Path("")
# All files and directories ending with .jpg (keep the *)
file_paths = sorted(destination_folder.glob("*.jpg"))
print(file_paths)

In [ ]:
if not file_paths:
    print("No files. Exiting.")
else:
    image_urls = [encode_image_to_base64(file_path) for file_path in file_paths]

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "system prompt"},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "user prompt",},
                    *[{"type": "image_url", "image_url": {"url": url}} for url in image_urls[::15]],
                ],
            }
        ],
        max_tokens=1000,
    )
    generated_markdown = response.choices[0].message.content
    # Copy the text to the clipboard
    pyperclip.copy(generated_markdown)
    print("-----------------------------------------------------")
print(generated_markdown)